In [43]:
import bioread
import np
import pandas as pd
import heartpy as hp
import neurokit2 as nk

acq_file = bioread.read_file(r'C:\Users\User\Downloads\80_28.1.24 (NPU_template).acq')

channels_to_extract = ['ECG - ECG100C', 'C5 - Expression' ]
data = {}
for channel_name in channels_to_extract:
    channel = acq_file.named_channels[channel_name]
    data[channel_name] = channel.data

df = pd.DataFrame(data)
print(df)


         ECG - ECG100C  C5 - Expression
0             0.000000              0.0
1             0.000153              0.0
2            -0.000153              0.0
3             0.000000              0.0
4             0.000000              0.0
...                ...              ...
3132788      -0.097656              0.0
3132789      -0.100098              0.0
3132790      -0.102539              0.0
3132791      -0.105438              0.0
3132792      -0.109253              0.0

[3132793 rows x 2 columns]


In [44]:
df = df.dropna()
df = df.reset_index(drop=True)


In [45]:
non_zero_indices = df.index[df['C5 - Expression'] != 0]
index_rows_to_change = []
for i in range(len(non_zero_indices)-1):
    if non_zero_indices[i+1] == non_zero_indices[i]+1:
        index_rows_to_change.append(non_zero_indices[i])
for index in index_rows_to_change:
    df.loc[index, 'C5 - Expression'] = 0

In [46]:
print(df)

         ECG - ECG100C  C5 - Expression
0             0.000000              0.0
1             0.000153              0.0
2            -0.000153              0.0
3             0.000000              0.0
4             0.000000              0.0
...                ...              ...
3132788      -0.097656              0.0
3132789      -0.100098              0.0
3132790      -0.102539              0.0
3132791      -0.105438              0.0
3132792      -0.109253              0.0

[3132793 rows x 2 columns]


In [47]:
df_baseline = df['ECG - ECG100C'][df['C5 - Expression']==99]

print(df_baseline)

481122    0.016327
Name: ECG - ECG100C, dtype: float64


In [48]:
df_baseline_minute = pd.DataFrame()
for index in df_baseline.index:
    df_slice = df['ECG - ECG100C'].iloc[index : index + 2000 * 60]
    df_slice = df_slice.to_frame(name='ECG - ECG100C')
    df_baseline_minute = pd.concat([df_baseline_minute, df_slice])
print(df_baseline_minute)

        ECG - ECG100C
481122       0.016327
481123       0.016632
481124       0.017548
481125       0.017548
481126       0.018463
...               ...
601117      -0.069733
601118      -0.070953
601119      -0.071259
601120      -0.072021
601121      -0.072632

[120000 rows x 1 columns]


In [49]:
baseline_minute_array = df_baseline_minute['ECG - ECG100C'].values.flatten()
print(baseline_minute_array)

[ 0.0163269   0.01663208  0.01754761 ... -0.07125854 -0.07202148
 -0.07263184]


In [55]:

signals = nk.ecg_process(baseline_minute_array, 2000)
heart_rate = nk.ecg_rate(signals)
print(heart_rate)

[26.53670043 26.09830361 28.30188679 29.16869227 27.63703363 27.95899348
 27.66251729 26.72605791 29.1403594  29.0275762  29.9550674  32.71537623
 30.58103976 23.83790226 25.86206897 28.70813397 28.11621368 27.07581227
 27.26033621 26.92998205 25.3164557  27.53556677 26.89376961 25.9965338
 26.80965147 24.73206925 24.52984464 26.39683238 28.30188679 31.69572108
 31.07198343 22.22222222 23.55712603 25.22068096 25.12562814 24.23263328
 25.45608825 24.62043496 24.74226804 26.78571429 24.600246   26.15518745
 27.76492365 24.35064935 24.15458937 29.73240833 29.71768202 23.82843527
 24.26202992 26.0756193  25.21008403 23.44665885]


In [51]:
ecg_data =  {
    "n_cue": [],
    "n_no_cue": [],
    "p_cue": [],
    "p_no_cue": [],
    "u_cue": [],
    "u_no_cue": []
}
for index, row in df.iterrows():
    value = row['C5 - Expression']
    ecg_value = row['ECG - ECG100C']
    
    
    if value == 020.0:
        ecg_data["n_cue"].append(ecg_value)
    if value == 030.0:
        ecg_data["n_no_cue"].append(ecg_value)
    if value == 120.0:
        ecg_data["p_cue"].append(ecg_value)
    if value == 130.0:
        ecg_data["p_no_cue"].append(ecg_value)
    if value == 220.0:
        ecg_data["u_cue"].append(ecg_value)
    if value == 230.0:
        ecg_data["u_no_cue"].append(ecg_value)
        
max_length = max(len(ecg_data[key]) for key in ecg_data)
for key in ecg_data:
    while len(ecg_data[key]) < max_length:
        ecg_data[key].append(None)
df_ecg = pd.DataFrame(ecg_data)
print(df_ecg)


    
         


       n_cue  n_no_cue     p_cue  p_no_cue     u_cue  u_no_cue
0  -0.048065 -0.008698 -0.004578  0.016174  0.102844 -0.026703
1  -0.019684  0.130157 -0.006866 -0.040588  0.067291 -0.094452
2  -0.026398 -0.010529  0.030975 -0.038300 -0.004425 -0.044556
3  -0.152893  0.074768 -0.047150 -0.041046 -0.025482 -0.048828
4  -0.028381  1.200714  0.029297 -0.042725 -0.053406  0.059814
5  -0.014648 -0.022430 -0.019989 -0.561829 -0.009155  0.146790
6   0.114136  0.104828       NaN -0.021210       NaN -0.041046
7  -0.020294  0.016479       NaN -0.087585       NaN -0.053253
8  -0.012054 -0.007782       NaN       NaN       NaN       NaN
9        NaN -0.016785       NaN       NaN       NaN       NaN
10       NaN  0.139313       NaN       NaN       NaN       NaN
11       NaN  0.013428       NaN       NaN       NaN       NaN


In [52]:
## trying with HEARTPY df
ecg_sampling_rate = 2000
mean_heart_rate = {}
working_data, measures = hp.process(df["ECG - ECG100C"].values, ecg_sampling_rate)
mean_heart_rate["ECG - ECG100C"] = measures['bpm']



In [53]:
measures

{'bpm': 55.370887615384696,
 'ibi': 1083.6019176136363,
 'sdnn': 96.3206670362669,
 'sdsd': 57.10036487168562,
 'rmssd': 92.14341368092913,
 'pnn20': 0.823445318084346,
 'pnn50': 0.5911365260900643,
 'hr_mad': 59.0,
 'sd1': 65.15455829273219,
 'sd2': 119.220180224025,
 's': 24403.06920781976,
 'sd1/sd2': 0.5465061214494153,
 'breathingrate': 0.30157408560770455}

In [54]:
## trying with HEARTPY
import numpy as np 

ecg_sampling_rate = 2000
mean_heart_rate = {}
for column in df_ecg.columns:
    nmpy = np.array(df_ecg[column])
    working_data, measures = hp.process(nmpy, ecg_sampling_rate)
    mean_heart_rate[column] = measures['bpm']
    mean_heart_rate = {column: sum(heart_rates) / len(heart_rates) for column, heart_rates in mean_heart_rate.items()}
print("Mean Heart Rates:")
for column, mean_hr in mean_heart_rate.items():
    print(f"{column}: {mean_hr} bpm")

ValueError: negative dimensions are not allowed

In [ ]:


## trying with neurokit   
ecg_sampling_rate = 2000
heart_rate_data = {}
for column in df_ecg.columns:
    signals = nk.ecg_process(df_ecg[column], ecg_sampling_rate)
    heart_rate = nk.ecg_rate(signals)
    heart_rate_data[column] = heart_rate
print("Heart Rates:")
for column, heart_rate in heart_rate_data.items():
    print(f"{column}: {heart_rate.mean()} bpm")

heart_rate_df = pd.DataFrame(heart_rate_data)

mean_heart_rate_per_column = heart_rate_df.mean(axis=0)

print("Mean Heart Rate per column:")
print(mean_heart_rate_per_column)
    
    







    

In [ ]:
nmpy